*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.crm_prd_info LIMIT 5;

In [0]:
df=spark.table("bara_slaes_project.bronze.crm_prd_info")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'prd_id':'Product_ID',
    'prd_key':'Product_Key',
    'prd_nm':'Product_name',
    'prd_cost':'Product_Cost',
    'prd_line':'Product_Line',
    'prd_start_dt':'Product_Start_Date',
    'prd_end_dt':'Product_End_Date'
}

df = df.withColumnsRenamed(rename_mapping)

In [0]:
## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

In [0]:
# *Replace Null Values*
median_Product_Cost = df.approxQuantile("Product_Cost", [0.5], 0.01)[0]
Replae_na_values={
    "Product_Cost":median_Product_Cost,
    "Product_Line":"Unknown"}
df=df.fillna(value=Replae_na_values)

In [0]:
# *Rename Values*
Product_Line_replacements = {
    "M":"Mountain",
    "R": "Road",
    "S": "Other Sales",
    "T": "Touring",
}
df=df.replace(Product_Line_replacements,subset=["Product_Line"])

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.silver.product_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.product_info LIMIT 5;